# Data Download

In [2]:
pwd

'/storage/yangjianLab/chenwenhao/projects/202312_GPS/src/GPS/docs/source'

In [17]:
%env PATH=/storage/yangjianLab/chenwenhao/Software/micromamba/envs/torch_v2/bin:/storage/yangjianLab/chenwenhao/Software/google-cloud-sdk/bin:/home/yangjianLab/chenwenhao/bin:/storage/yangjianLab/chenwenhao/bin:/opt/slurm/sbin:/opt/slurm/bin:/soft/singularity-3.7.1/bin:/storage/yangjianLab/chenwenhao/Software/micromamba/condabin:/storage/yangjianLab/chenwenhao/Software/google-cloud-sdk/bin:/home/yangjianLab/chenwenhao/bin:/storage/yangjianLab/chenwenhao/bin:/opt/slurm/sbin:/opt/slurm/bin:/home/yangjianLab/chenwenhao/.cargo/bin:/home/yangjianLab/chenwenhao/bin/bin:/home/yangjianLab/chenwenhao/.local/bin:/home/yangjianLab/chenwenhao/bin:/opt/slurm/sbin:/opt/slurm/bin:/soft/modules/modules-4.7.0/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/home/yangjianLab/chenwenhao/bin

env: PATH=/storage/yangjianLab/chenwenhao/Software/micromamba/envs/torch_v2/bin:/storage/yangjianLab/chenwenhao/Software/google-cloud-sdk/bin:/home/yangjianLab/chenwenhao/bin:/storage/yangjianLab/chenwenhao/bin:/opt/slurm/sbin:/opt/slurm/bin:/soft/singularity-3.7.1/bin:/storage/yangjianLab/chenwenhao/Software/micromamba/condabin:/storage/yangjianLab/chenwenhao/Software/google-cloud-sdk/bin:/home/yangjianLab/chenwenhao/bin:/storage/yangjianLab/chenwenhao/bin:/opt/slurm/sbin:/opt/slurm/bin:/home/yangjianLab/chenwenhao/.cargo/bin:/home/yangjianLab/chenwenhao/bin/bin:/home/yangjianLab/chenwenhao/.local/bin:/home/yangjianLab/chenwenhao/bin:/opt/slurm/sbin:/opt/slurm/bin:/soft/modules/modules-4.7.0/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/home/yangjianLab/chenwenhao/bin


In [21]:
%%bash
GPS -h

usage: GPS [-h] [--version]
           {run_find_latent_representations,run_latent_to_gene,run_generate_ldscore,run_spatial_ldsc,run_cauchy_combination,run_visualize,run_all_mode}
           ...

 GPS: Genetics-informed pathogenic spatial mapping

options:
  -h, --help            show this help message and exit
  --version, -v         show program's version number and exit

Available subcommands:
  {run_find_latent_representations,run_latent_to_gene,run_generate_ldscore,run_spatial_ldsc,run_cauchy_combination,run_visualize,run_all_mode}
                        Subcommands
    run_find_latent_representations
                        Run Find_latent_representations 
                        Find the latent representations of each spot by running GNN-VAE
    run_latent_to_gene  Run Latent_to_gene 
                        Find gene marker gene scores for each spot by using latent representations from nearby spots
    run_generate_ldscore
                        Run Generate_ldscore 
        

In [18]:
!which python

/storage/yangjianLab/chenwenhao/Software/micromamba/envs/torch_v2/bin/python


In [19]:
!GPS

usage: GPS [-h] [--version]
           {run_find_latent_representations,run_latent_to_gene,run_generate_ldscore,run_spatial_ldsc,run_cauchy_combination,run_visualize,run_all_mode}
           ...

 GPS: Genetics-informed pathogenic spatial mapping

options:
  -h, --help            show this help message and exit
  --version, -v         show program's version number and exit

Available subcommands:
  {run_find_latent_representations,run_latent_to_gene,run_generate_ldscore,run_spatial_ldsc,run_cauchy_combination,run_visualize,run_all_mode}
                        Subcommands
    run_find_latent_representations
                        Run Find_latent_representations 
                        Find the latent representations of each spot by running GNN-VAE
    run_latent_to_gene  Run Latent_to_gene 
                        Find gene marker gene scores for each spot by using latent representations from nearby spots
    run_generate_ldscore
                        Run Generate_ldscore 
        